In [1]:
## Description: Impact of management training study
# Random assignment of maangers into or not into management training
# Measure the outcome of engagement of employees from the managers
# Some managers received the training even though not assigned to it - randomization not valid

## Sources: Causal Inference in Python


In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from patsy import dmatrix
from joblib import Parallel, delayed # for parallel processing
from toolz import partial
from dowhy import CausalModel

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\arsep\anaconda3\envs\ds1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# treatment is intervention, outcome is engagement_score, rest of vars are covariates
# reduce or even eliminate the bias when estimating the causal relationship between management training and employee engagement
df = pd.read_csv("management_training.csv")
df.head()

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size
0,76,1,0.277359,6,4,2,4,0.614261,0.224077,843
1,76,1,-0.449646,4,8,2,4,0.069636,0.224077,843
2,76,1,0.769703,6,4,2,4,0.866918,0.224077,843
3,76,1,-0.121763,6,4,2,4,0.029071,0.224077,843
4,76,1,1.526147,6,4,1,4,0.589857,0.224077,843


In [4]:
df.shape

In [5]:
df.dtypes

departament_id             int64
intervention               int64
engagement_score         float64
tenure                     int64
n_of_reports               int64
gender                     int64
role                       int64
last_engagement_score    float64
department_score         float64
department_size            int64
dtype: object

In [6]:
# df['intervention'] = df['intervention'].astype('str')
df['intervention'] = df['intervention'].astype('bool')

In [7]:
df['engagement_score'].describe()

count    1.039100e+04
mean     3.829313e-17
std      1.000000e+00
min     -3.111431e+00
25%     -7.033152e-01
50%     -3.449486e-02
75%      6.724723e-01
max      3.563639e+00
Name: engagement_score, dtype: float64

#### Naive Estimate of Average Treatment Effect (ATE)

In [8]:
treatment_avg = df.loc[df['intervention'] == 1,'engagement_score'].mean()
control_avg = df.loc[df['intervention'] == 0,'engagement_score'].mean()

print('Treatment Average:', treatment_avg.round(4))
print('Control Average:',  control_avg.round(4))
print('Treatment Effect:', (treatment_avg - control_avg).round(4))

Treatment Average: 0.1999
Control Average: -0.2347
Treatment Effect: 0.4346


#### Linear Regression Modeling

In [9]:
df['intervention'] = df['intervention'].astype('int')

In [10]:
# simple model comparing treatment vs. control
smf.ols("engagement_score ~ intervention",data=df).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-0.2347,0.014,-16.619,0.000,-0.262,-0.207
intervention,0.4346,0.019,22.616,0.000,0.397,0.472


In [11]:
# full regression model
# much smaller ATE than previous simple model
model = smf.ols("""engagement_score ~ intervention + tenure + last_engagement_score + department_score + n_of_reports + C(gender) + C(role)""", data=df).fit()

print("ATE:", model.params["intervention"])
print("95% CI:", model.conf_int().loc["intervention", :].values.T)

ATE: 0.26779085766768534
95% CI: [0.23357751 0.30200421]


#### Propensity Score Weighting

In [12]:
# propensity score model
ps_model = smf.logit("""intervention ~ tenure + last_engagement_score + department_score + C(n_of_reports) + C(gender) + C(role)""", data=df).fit(disp=0)

In [13]:
# propensity scores
data_ps = df.assign(propensity_score = ps_model.predict(df),)
data_ps[["intervention", "engagement_score", "propensity_score"]].head()

,intervention,engagement_score,propensity_score
0,1,0.277359,0.596106
1,1,-0.449646,0.391138
2,1,0.769703,0.602578
3,1,-0.121763,0.580990
4,1,1.526147,0.619976


In [14]:
data_ps.shape

In [15]:
# adding propensity_score var to simple regression model to avoid adjusting for the confounders of X
# ATE is very similar to full model
model = smf.ols("engagement_score ~ intervention + propensity_score",data=data_ps).fit()
model.params["intervention"]

#### Propensity Score Matching

In [16]:


T = "intervention"
X = "propensity_score"
Y = "engagement_score"

treated = data_ps.query(f"{T}==1")
untreated = data_ps.query(f"{T}==0")

# KNeighborsRegressor regression based on KNN
# find the nearest 1 n_neighbors match, KNN model with propensity_score as only feature to predict engagement_score

# control data model
mt0 = KNeighborsRegressor(n_neighbors=1).fit(untreated[[X]],untreated[Y])

# treatment data model
mt1 = KNeighborsRegressor(n_neighbors=1).fit(treated[[X]],treated[Y])

predicted = pd.concat([
    # find matches for the treated looking at the untreated knn model, match is the prediction of engagement_score for each row
    treated.assign(match=mt0.predict(treated[[X]])), # assign new cols to df (match col)
    
    # find matches for the untreated looking at the treated knn model
    untreated.assign(match=mt1.predict(untreated[[X]]))
])

predicted.head()

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size,propensity_score,match
0,76,1,0.277359,6,4,2,4,0.614261,0.224077,843,0.596106,0.557680
1,76,1,-0.449646,4,8,2,4,0.069636,0.224077,843,0.391138,-0.952622
2,76,1,0.769703,6,4,2,4,0.866918,0.224077,843,0.602578,-0.618381
3,76,1,-0.121763,6,4,2,4,0.029071,0.224077,843,0.580990,-1.404962
4,76,1,1.526147,6,4,1,4,0.589857,0.224077,843,0.619976,0.000354


In [17]:
predicted.loc[predicted['engagement_score'] == predicted['engagement_score'][2],:]
# predicted.head()
# predicted['engagement_score'].head()

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size,propensity_score,match
2,76,1,0.769703,6,4,2,4,0.866918,0.224077,843,0.602578,-0.618381


In [18]:
predicted.loc[predicted['match'] == predicted['engagement_score'][2],:]

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size,propensity_score,match
4993,63,0,1.111142,6,2,2,4,0.956035,0.196662,389,0.602594,0.769703
5698,58,0,0.180531,6,4,1,1,-0.349523,-0.946657,1732,0.602602,0.769703
7128,37,0,-0.618381,6,4,1,1,-0.173646,-0.045722,530,0.602567,0.769703


In [19]:
# Every unit is matched, now find estimated ATE
# (Y - Y_Predicted) * Intervention + (Y_Predicted - Y) * (1 - Intervention)
np.mean((predicted[Y] - predicted["match"])*predicted[T] + (predicted["match"] - predicted[Y])*(1-predicted[T]))

#### Inverse Propensity Weighting

In [20]:
weight_t = 1/data_ps.query("intervention==1")["propensity_score"]
weight_nt = 1/(1-data_ps.query("intervention==0")["propensity_score"])
t1 = data_ps.query("intervention==1")["engagement_score"] 
t0 = data_ps.query("intervention==0")["engagement_score"] 

y1 = sum(t1*weight_t)/len(data_ps)
y0 = sum(t0*weight_nt)/len(data_ps)

print("E[Y1]:", y1)
print("E[Y0]:", y0)
print("ATE", y1 - y0)

E[Y1]: 0.11656317232946753
E[Y0]: -0.1494155364781441
ATE 0.26597870880761165


In [21]:
# Alternative ATE calculation
np.mean(data_ps["engagement_score"] * (data_ps["intervention"] - data_ps["propensity_score"]) / (data_ps["propensity_score"]*(1-data_ps["propensity_score"])))

In [22]:
# Unfortunately, computing the standard error for IPW is not as straightforward as with linear regression. 
# The most straightforward way to obtain a confidence interval around your IPW estimate is by using the bootstrap method. 
# With this method, you will repeatedly resample the data with replacement to obtain multiple IPW estimators. 
# You can then calculate the 2.5th and 97.5th percentiles of these estimates to obtain a 95% confidence interval.

In [23]:
# define function that computes the IPW estimator
def est_ate_with_ps(df, ps_formula, T, Y):
    
    X = dmatrix(ps_formula, df)
    ps_model = LogisticRegression(penalty="l2",max_iter=1000).fit(X, df[T])
    ps = ps_model.predict_proba(X)[:, 1]
    
    # compute the ATE
    return np.mean((df[T]-ps) / (ps*(1-ps)) * df[Y]) 

In [24]:
formula = """tenure + last_engagement_score + department_score+ C(n_of_reports) + C(gender) + C(role)""" 
T = "intervention"
Y = "engagement_score"

est_ate_with_ps(df, formula, T, Y)

In [25]:
def bootstrap(data, est_fn, rounds=200, seed=123, pcts=[2.5, 97.5]):
    np.random.seed(seed)
    
    stats = Parallel(n_jobs=4)(
        delayed(est_fn)(data.sample(frac=1, replace=True))
        for _ in range(rounds)
    )
    
    return np.percentile(stats, pcts)

In [26]:
def addNumber(x, number):
    return x + number

add2 = partial(addNumber, number=2)
add4 = partial(addNumber, number=4)

In [27]:
# add2(3)
add4(3)

In [28]:
from toolz import partial

print(f"ATE: {est_ate_with_ps(df, formula, T, Y)}")

est_fn = partial(est_ate_with_ps, ps_formula=formula, T=T, Y=Y)
print(f"95% C.I.: ", bootstrap(df, est_fn))

ATE: 0.2662350402507956
95% C.I.:  [0.22654202 0.30087918]
